In [ ]:
from saliencyserieslab.plotting import plot_graph
import pickle


with open('../data/ECG200.pkl', 'rb') as f:
    data = pickle.load(f)

plot_graph(data[0][0])

with open('../data/Plane.pkl', 'rb') as f:
    data = pickle.load(f)

plot_graph(data[0][0])

with open('../data/SwedishLeaf.pkl', 'rb') as f:
    data = pickle.load(f)

plot_graph(data[0][0])